In [1]:
!pip3 install tensorflow-hub  tensorflow-text pandas numpy tensorflow

In [1]:
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf


: 

In [8]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [9]:
bert_preprocess_model = hub.KerasLayer(preprocess_url) # l1

In [10]:
text_test = ['nice movie indeed','I love python programming']
text_preprocessed = bert_preprocess_model(text_test)
text_preprocessed.keys()

dict_keys(['input_word_ids', 'input_mask', 'input_type_ids'])

Note : CLS and Separator are used in start and end of the sentence

*input_word_ids -> word ids
*input_mask -> masking words
*input_type_ids -> type ids


In [11]:
bert_model = hub.KerasLayer(encoder_url) # l2
text_results = bert_model(text_preprocessed)
text_results.keys()

dict_keys(['encoder_outputs', 'sequence_output', 'default', 'pooled_output'])

In [14]:
import pandas as pd

df = pd.read_csv('/content/spam.csv', delimiter=',');

df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [15]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [16]:
df_spam = df[df['Category'] == 'spam']
df_ham = df[df['Category'] == 'ham']

In [17]:
df_ham.count()

Category    4825
Message     4825
dtype: int64

In [18]:
df_balanced = pd.concat([df_spam, df_ham[:len(df_spam)]]) # downgrading

In [19]:
df_balanced['Category'].value_counts()

spam    747
ham     747
Name: Category, dtype: int64

In [20]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit label encoder and transform categories in the "category" column
df_balanced['category_numeric'] = label_encoder.fit_transform(df_balanced['Category'])
df_balanced['category_numeric'].unique()

# spam -> 0, ham-> 1

array([1, 0])

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['category_numeric'], stratify=df_balanced['category_numeric'])

In [22]:
# Now lets import BERT model and get embeding vectors for few sample statements
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

Certainly! Let's break down and add a description for the provided function `get_sentence_embedding`:


Explanation:

- `sentences`: This parameter is a list of strings representing sentences for which you want to obtain embeddings.
  
- `preprocessed_text`: This variable stores the preprocessed text obtained by applying some preprocessing steps, likely tokenization, and other necessary transformations required by the BERT model. This could include converting the list of sentences into tokenized input suitable for the BERT model.

- `sentence_encoding`: This variable holds the encoded representations of the sentences. It's obtained by passing the preprocessed text through the BERT model (`bert_encoder`), likely extracting the pooled output, which is a fixed-size representation of the entire input sequence. This step captures the contextual information of each sentence.

- `return sentence_encoding`: This function returns the computed sentence embeddings, which are numerical representations of the input sentences obtained from the pre-trained BERT model. These embeddings can be further used for downstream tasks such as sentiment analysis, text classification, or similarity comparison between sentences.

This code snippet demonstrates how to use the `get_sentence_embedding` function to obtain sentence embeddings for a list of sentences. The resulting `embeddings` variable will contain the numerical representations of the input sentences, which can then be used for various natural language processing tasks.

In [23]:
def get_sentence_embeding(sentences: list = []):
  preprocessed_text = bert_preprocess(sentences)
  sentence_encoding = bert_encoder(preprocessed_text)['pooled_output']
  return sentence_encoding

res=get_sentence_embeding(
    ["banana",
     "grapes",
     "mango",
     "elon musk",
     "jeff bezos"]
)


In [24]:
from sklearn.metrics.pairwise import cosine_similarity
# check similarity
cosine_similarity([res[3]], [res[4]])

array([[0.9872036]], dtype=float32)

In [25]:
# creating model in tensorflow
# 1. Sequential

# 2. Functional

This code snippet represents the construction of a neural network model using BERT (Bidirectional Encoder Representations from Transformers) layers for natural language processing tasks. Here's a breakdown of what each section does:

1. **BERT Layers Setup**:
   - `text_input`: This line defines an input layer for text data. It specifies the shape of the input data (an empty shape, meaning the input can have variable length), data type (string), and a name for this input layer.
   - `preprocessed_text`: This line preprocesses the input text using a BERT preprocessing function (`bert_preprocess`). This function likely performs tokenization, padding, and other necessary preprocessing steps specific to BERT.
   - `outputs`: This line passes the preprocessed text through a BERT encoder (`bert_encoder`) to obtain BERT model outputs.

2. **Neural Network Layers**:
   - `l`: This line applies a dropout layer with a dropout rate of 0.1 to the pooled output obtained from the BERT encoder. Dropout is a regularization technique used to prevent overfitting by randomly setting a fraction of input units to zero during training.
   - Another dense layer with a single neuron and a sigmoid activation function is added. This layer is typically used for binary classification tasks, where the output value represents the probability of the positive class.

3. **Final Model Construction**:
   - `model`: This line constructs the final model using the input layer (`text_input`) and the output layer (`l`). It defines a `tf.keras.Model` object that takes the text input and produces the final output.



The sigmoid activation function is commonly used in binary classification tasks, where the goal is to predict one of two possible outcomes (e.g., positive/negative, spam/not spam, etc.). Here's why the sigmoid activation function is suitable for such tasks:

1. **Output Interpretation**: The sigmoid function maps the output of the neural network to a value between 0 and 1, which can be interpreted as a probability. In binary classification, this probability represents the likelihood or confidence of the input belonging to the positive class.

2. **Probabilistic Interpretation**: The sigmoid function outputs probabilities that sum up to 1. This property is useful for probabilistic interpretations, such as setting a threshold (e.g., 0.5) to decide the predicted class based on whether the output probability is above or below the threshold.

3. **Differentiability**: The sigmoid function is differentiable everywhere, which is important for training the neural network using gradient-based optimization algorithms like backpropagation. This allows the model to learn the appropriate parameters by adjusting them based on the error between predicted and actual values.

4. **Smooth Transition**: The sigmoid function has a smooth and continuous curve, which ensures smooth transitions in the output probabilities as the input changes. This property can lead to more stable and consistent model predictions.

Overall, the sigmoid activation function is well-suited for binary classification tasks because of its probabilistic interpretation, differentiability, and smooth transition properties.

In [26]:
# Binary class -> Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
# # Multi class -> Bert layers
# text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
# preprocessed_text = bert_preprocess(text_input)
# outputs = bert_encoder(preprocessed_text)

# # Neural network layers
# l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
# l = tf.keras.layers.Dense(num_classes, activation='softmax', name="output")(l)  # num_classes is the number of classes in your dataset

# # Use inputs and outputs to construct a final model
# model = tf.keras.Model(inputs=[text_input], outputs=[l])


In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer_2 (KerasLayer)  {'input_mask': (None, 128)   0         ['text[0][0]']                
                             , 'input_type_ids': (None,                                           
                              128),                                                               
                              'input_word_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [28]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 608s 17s/step - loss: 0.6579 - accuracy: 0.6179 - precision: 0.6218 - recall: 0.6018
Epoch 2/10
24/35 [===================>..........] - ETA: 3:02 - loss: 0.5492 - accuracy: 0.7812 - precision: 0.7813 - recall: 0.7731

In [ ]:
model.save("text_classification")

In [ ]:
model = tf.keras.models.load_model("text_classification")

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

In [ ]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)